In [1]:
from Naive_bayes import NaiveBayes
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import distance 
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
from time import sleep
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import math as mt
import time
from numpy.linalg import inv, det
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from numba import jit
import numpy as np
%matplotlib inline

In [2]:
mfeat = pd.read_csv("mfeat/mfeat-fac", delimiter="\0", sep="\0", header=None )
mfeatFou = pd.read_csv("mfeat/mfeat-fou", delimiter="\0", sep="\0", header=None )
mfeatka = pd.read_csv("mfeat/mfeat-kar", delimiter="\0", sep="\0", header=None )
y = pd.read_csv("results/fuzzy_crisp_partition_572953998.csv",sep="\n", header=None)
y = y.values
y  = [y[i][0] for i in range(len(y))]
y = np.array(y)

In [3]:
#Retirando os espaços. Transformando da elemento em uma lista com valores númericos, Atribuindo isso a uma 
# lista com tudo processado e normaliza
def preprocessing(data):
    new_data = []
    for ex in data:
        ex = ex[0].split(" ")
        exemplos1 = []
        for element in ex:
            if element != "":
                exemplos1.append(element)
        new_data.append(exemplos1) 
    
    ## Normaliza
  
    new_data = np.array(new_data).astype(float)
    scaler = StandardScaler()
    scaler.fit(new_data)
    new_data = scaler.transform(new_data)
    return new_data
   

In [4]:
mfeat =  preprocessing(mfeat.values)
mfeatFou =  preprocessing(mfeatFou.values)
mfeatKa =  preprocessing(mfeatka.values)

In [5]:
kf = KFold(n_splits=10,shuffle=True)

In [6]:
from knn import KNN

In [7]:
k = KNN()

In [8]:
ks = [5,13, 3]

In [9]:
kf = KFold(n_splits=10,shuffle=True)

In [10]:

list_acc_n = []
list_acc_k = []
count = 0


for i in trange(0, 30):
    acc_fold_n = []
    acc_fold_k = []
    for train_index, test_index in kf.split(np.array((mfeat))):
       
        X_trains = []
        y_train_views = []
        X_tests = []
    
        X_train_view1, X_test_view1 = mfeat[train_index], mfeat[test_index]
        y_train_view1, y_test_view1 = y[train_index], y[test_index]
      
       
       
        X_train_view2, X_test_view2 = mfeatFou[train_index], mfeatFou[test_index]
        y_train_view2, y_test_view2 = y[train_index], y[test_index]
       
   

        X_train_view3, X_test_view3 = mfeatKa[train_index], mfeatKa[test_index]
        y_train_view3, y_test_view3 = y[train_index], y[test_index]
        
  
        
        ##Values   
        X_trains = [X_train_view1, X_train_view2, X_train_view3]
        y_train_views = [y_train_view1, y_train_view2, y_train_view3]
        X_tests = [X_test_view1, X_test_view2, X_test_view3]
        nb = NaiveBayes()
        
        nb.fit(X_trains, y_train_views)
        
        y_pred_n = nb.predict(X_tests=X_tests);
        
        acc_fold_n = accuracy_score(y_test_view1, y_pred_n)
        
        k = KNN()
        
        k.fit(X_trains=X_trains, y_trains=y_train_views, k=ks)
        
        y_pred_k = k.predict(X_tests=X_tests);
        
        acc_fold_k = accuracy_score(y_test_view1, y_pred_k)
        
    list_acc_n.append(np.mean(acc_fold_n))
    list_acc_k.append(np.mean(acc_fold_k))

100%|██████████| 30/30 [3:06:32<00:00, 469.16s/it]  


In [11]:
np.mean(list_acc_n)

0.673

In [12]:
np.mean(list_acc_k)

0.6978333333333334

In [14]:
acc_knn = pd.DataFrame(list_acc_k, columns=['acc'])
acc_knn_csv = acc_knn.to_csv()
arq = open("results/accuracy_knn_cros", "w")
arq.write(acc_knn_csv)
arq.close()

In [15]:
acc_naive = pd.DataFrame(list_acc_n, columns=['acc'])
acc_naive_csv = acc_naive.to_csv()
arq = open("results/accuracy_naive_cros", "w")
arq.write(acc_naive_csv)
arq.close()

In [16]:
acc_naive = pd.read_csv("results/accuracy_naive_cros")

In [17]:
acc_naive = acc_naive['acc'].values

In [18]:
from scipy.stats import wilcoxon

In [20]:
test = wilcoxon(acc_naive, list_acc_k)

In [21]:
test.pvalue < 0.05

True